# Amazon Bedrock による質問応答

> *このノートブックは、SageMaker Studioの **`Data Science 3.0`** カーネルで実行してください*

## 概要

質問応答は、自然言語で提示されたクエリに対する回答を抽出する重要なタスクです。通常、質問応答システムは、構造化データまたは非構造化データを含むナレッジベースに対してクエリを実行し、そこから正確な情報を含む応答を生成します。企業利用ケースにおいて特に、高い精度を確保することが、有用で信頼できる質問応答システムを開発するための鍵となります。 

Amazon Titan や Claude のような生成 AI モデルは、確率分布を使用して問題への応答を生成します。これらのモデルは膨大な量のテキストデータに基づいてトレーニングされているため、シーケンスで次に来るものや、特定の単語の後に続く単語を予測できます。ただし、データには常にある程度の不確実性があるため、これらのモデルはすべての問題に対して正確または決定論的な回答を提供することはできません。企業は、分野特有のデータ、企業独自のデータ、さらに一般的にはモデルのトレーニングを受けていないデータを照会する必要があります。彼らは LLM にその情報を使って彼らの問題に答えてほしいと思っています。

このラボでは、Amazon Titan Text モデルを使用してクエリに応答する方法を学習します。

最初にコンテキストなしでモデルを実行し、次にモデルに送信されるプロンプトに手動でコンテキストを追加します。このアプローチを Retrieval Augmented Generation (RAG) と混同しないでください。この方法は、短い文書やシングルトン・アプリケーションに適しており、プロンプトに収まらないような大きな文書を使用することが多い、企業レベルの質問応答システムにスケールすることはできません。

### 課題
- 質問に対して、事実基づく回答を返すには、モデルをどのように設定すべきか

### 提案
上記課題に対し、このノートブックでは次のような戦略をとります。

#### ドキュメントの準備
質問に答える前に、ドキュメントを処理してドキュメントストアインデックスに格納する必要があります。
- ここでは、関連する全コンテキストをリクエストに含めてモデルに送信し、応答を期待します。

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys

import boto3
import botocore


module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww


# ---- ⚠️ AWS 環境の設定に応じて、以下の行のコメントを外して編集してください。 ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## セクション1: モデルの知識を用いた Q&A
このセクションでは、Bedrock サービスによって提供されるモデルを使用して、トレーニングフェーズ中に獲得した知識に基づいて質問に答えることを試みます

このノートブックでは、Amazon Bedrock クライアントの `invoke_model()` メソッドを使用します。このメソッドを使用するために必要な必須パラメータは、Amazon Bedrock モデルの ARN を表す `modelId` と、タスクのプロンプトである `body` です。`body` プロンプトは、選択した基盤モデルプロバイダーによって異なります。以下で詳細を説明します:


```
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

## シナリオ
タイヤ交換の情報をモデルに提供する方法が必要です。まず、トレーニングデータに基づいて、特定のメーカーと車種について回答を提供するようモデルに尋ねます。この手法は「ゼロショット」と呼ばれます。すぐにわかるのは、モデルは関連性のある回答を返しているように見えるものの、実際にはハルシネーションを起こしていることです。その理由は、架空の車について尋ねたときに、ほぼ同じ回答が返ってくるためです。

この状況は、特定のメーカーと車種に関する追加のデータでモデルのトレーニングを増強する必要があることを意味しています。そうするとモデルは非常に具体的な回答を返してくれます。このノートブックでは外部ソースを使用してデータを増強するのではなく、RAG ベースの増強システムの動作をシミュレートします。

最終テストのために、特定の車のタイヤ交換方法を説明しているマニュアルから詳細なセクションを提供します。そして、モデルから練られた応答が返ってくるかテストします。

## タスク
プロセスを開始するには、Bedrock で提供されるモデルの 1 つを選択します。このユースケースでは、Titan を選択します。このモデルは、車に関する一般的な質問に回答できます。

例えば、Titan モデルに Audi のパンクタイヤの交換方法を教えてもらいます。

In [ ]:
prompt_data = """You are an helpful assistant. Answer questions in a concise way. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my Audi A8?
Answer:"""
parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":1,
    "topP":1
    }

#### モデルを呼び出してJSON本文を渡し、応答を生成します

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1" #
accept = "application/json"
contentType = "application/json"
try:
    
    response = boto3_bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body.get("results")[0].get("outputText")
    print_ww(answer.strip())

except botocore.exceptions.ClientError as error:
    if  error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

モデルは車のパンクタイヤ交換のプロセスを説明する回答を提供できますが、同じ説明は任意の車に有効な可能性があります。残念ながらこれは、スペアタイヤを持たない Audi A8\ に対する正しい回答ではありません。これは、モデルが車のタイヤ交換の指示を含むデータで訓練されたためです。

この問題の別の例は、完全に架空の車のブランドとモデル、例えば Amazon Tirana に同じ質問をしてみることで確認できます。

In [ ]:
prompt_data = "How can I fix a flat tire on my Amazon Tirana?"
body = json.dumps({"inputText": prompt_data, 
                   "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1" # 使用する環境に合わせて、適切なモデルバージョンを指定してください。
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print_ww(answer.strip())

この問題をどのように修正し、車種固有の正しい指示に基づいて回答を提供するようにモデルをさせることができるでしょうか。

Facebook の 2020 年の研究では、プロンプトの一部として追加のナレッジベースを提供することで、LLM の知識をオンザフライで増強できることがわかりました。このアプローチは、検索拡張生成 (RAG) と呼ばれます。

この方法を使ってアプリケーションを改善する方法を見ていきましょう。

以下は Audi A8 のマニュアルの抜粋です (実際には本物のマニュアルではありませんが、そうだと想定します)。このドキュメントは、Titan Text モデルのプロンプトに完全に収まる程度に短く便利です。

(マニュアル抜粋)

```
Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact Rivesla or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible.

```

次に、このテキストを元の質問と一緒にプロンプトに「埋め込みます」。プロンプトは、モデルにコンテキストとして提供された情報をヒントとして参照するように構築されています。

In [ ]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact Audi or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

#### マニュアルの抜粋全文と質問をモデルに渡す

In [ ]:
question = "How can I fix a flat tire on my Audi A8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide.
#
{context}
#

Question: {question}
Answer:"""

##### Boto3 を使用してモデルを実行し、応答を生成

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1" #amazon.titan-tg1-large"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print_ww(answer.strip())

モデルはコンテキストを理解して関連する回答を生成するのに時間がかかるため、ユーザーは数秒間応答を待たなければならず、ユーザー体験が低下する可能性があります。

Bedrock はモデルがトークンを生成している間に、サービスの出力を生成するストリーミング機能をサポートしています。これを行う方法の例を以下に示します。

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

## まとめ
応答は、タイヤの交換方法を要約した段階的な説明になっています。このシンプルな例は、より複雑な応答を生成するために `RAG` (検索拡張生成) を活用できることを示しています。